# Interactive Occlusion Heatmap slider for all models

For a given patient id the occlusion heatmap is calculated for all models and displayed in a slider.
The size of the occlusion can be adjusted manually.

### Imports

In [ ]:
%matplotlib inline

import os
import h5py
import tqdm
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.metrics import confusion_matrix, roc_curve, auc

import tensorflow as tf
from tensorflow import keras

print("TF  Version",tf.__version__)

In [ ]:
# check and set path before loading modules
print(os.getcwd())
INPUT_DIR = "/tf/notebooks/schnemau/xAI_stroke_3d/"
OUTPUT_DIR = "/tf/notebooks/bule/explainable_AI/"
if os.getcwd() != OUTPUT_DIR:
    os.chdir(OUTPUT_DIR)

In [ ]:
import functions_model_definition as md
import functions_read_data as rdat
import functions_occlusion as oc
import functions_gradcam as gc
import functions_plot_heatmap as phm
import functions_slider as sl

### Load Data and Set Parameters

In [ ]:
# Define Version
version = "10Fold_CIBLSX" # one of:
# 10Fold_sigmoid_V0, 10Fold_sigmoid_V1, 10Fold_sigmoid_V2, 10Fold_sigmoid_V2f, 10Fold_sigmoid_V3
# 10Fold_softmax_V0, 10Fold_softmax_V1, andrea
# 10Fold_CIB, 10Fold_CIBLSX

# Define Model Version
model_version = 1

# define weighting
hm_mode = "wgt" 

# define heatmap type
hm_type = "gc"
pred_hm_only = False

# define paths
DATA_DIR, WEIGHT_DIR, DATA_OUTPUT_DIR, PIC_OUTPUT_DIR, pic_save_name = rdat.dir_setup(
    INPUT_DIR, OUTPUT_DIR, version, model_version, weight_mode = hm_mode,
    hm_type = hm_type, ending = "_predcl" if pred_hm_only else "_bothcl")

In [ ]:
## load images and ids
(X_in, pat_ids, id_tab, all_results_tab, pat_orig_tab, pat_norm_tab, num_models) = rdat.version_setup(
    DATA_DIR = DATA_DIR, version = version, model_version = model_version)

## Model

In [ ]:
# define model
(input_dim_img, output_dim, LOSS, layer_connection, last_activation) = md.model_setup(version)

model_3d = md.model_init(
    version = version, 
    output_dim = output_dim,
    LOSS = LOSS,
    layer_connection = layer_connection,
    last_activation = last_activation,
    C = 2,
    learning_rate = 5*1e-5,
    batch_size = 6,
    input_dim = input_dim_img,
    input_dim_tab = pat_norm_tab.drop(columns=["p_id"]).shape[1] if "LSX" in version else None,
)

In [ ]:
# Define Model Name
generate_model_name = md.set_generate_model_name(
    model_version = model_version, 
    layer_connection = layer_connection, 
    last_activation = last_activation, 
    path = WEIGHT_DIR)  

# Occlusion without slider

In [ ]:
(res_table, res_images, res_model_names) = gc.get_img_and_models(
        [3], results = all_results_tab, pats = pat_ids, imgs = X_in, 
        gen_model_name = generate_model_name,
        num_models = num_models)

In [ ]:
res_table

Two different occlusions are recommended.  
However, the occlusion can also be defined by the user. The third row of the following output must be all 0 and the fourth row must be True and the occlusion will work.

In [ ]:
### occlusion
# occ_size = (16, 16, 12)
# occ_stride = 8
# occ_size = (14, 14, 10)
# occ_stride = (6)
occ_size = (18, 18, 4)
occ_stride = (10, 10, 3)
print("number of occlusions: ", int(np.prod(((np.array(res_images.shape[1:4]) - occ_size) / occ_stride) + 1)))
print("number of occlusions per axis: ", ((np.array(res_images.shape[1:4]) - occ_size) / occ_stride) + 1)
print((np.asarray(res_images.shape[1:4]) - occ_size) % occ_stride) # all must be zero
print(all(np.array(occ_size) > occ_stride)) # must be true
print(np.array(res_images.shape[1:4]) / occ_size) # if all same, then same ratio of occ_size to image size

In [ ]:
import time
from importlib import reload
reload(oc)

start = time.time()
(heatmap, resized_img, max_hm_slice, hm_mean_std, all_heatmaps) =  oc.volume_occlusion(
    volume = res_images, 
    res_tab = res_table, 
    tabular_df=pat_norm_tab,
    occlusion_size = np.array(occ_size), 
    normalize = False,
    both_directions = False,
    invert_hm = "pred_class",
    cnn = model_3d,
    model_names = res_model_names[0][:], # select model
    occlusion_stride = occ_stride)

end = time.time()
print(end - start)

In [ ]:
phm.plot_heatmap(np.squeeze(resized_img, axis=-1), np.squeeze(heatmap, axis=-1),
                version = "overlay",
                mode = "avg",
                hm_colormap="jet",
                hm_positive=True,
                colorbar=True,
                )

In [ ]:
phm.plot_heatmap(np.squeeze(resized_img, axis=-1), np.squeeze(heatmap, axis=-1),
                version = "overlay",
                mode = "def",
                slices = (50,20,20),
                hm_colormap="jet",
                hm_positive=True,
                colorbar=True,
                add_orientation=True)

Change tabular part 

In [ ]:
pat_norm_tab_cp = pat_norm_tab.copy()

In [ ]:
res_table.y_pred_trafo_avg

In [ ]:
def change_input_tabular(tabular_df, value, column = "age", orig_mean = 67.61277641277641, orig_sd = 15.215466719566002):
    tabular_df_cp = tabular_df.copy()
    tabular_df_cp[column] = ( value - orig_mean ) / orig_sd
    return tabular_df_cp

In [ ]:
pat_norm_tab_cp = change_input_tabular(pat_norm_tab_cp, 40, "age")

In [ ]:
import functions_metrics as fm

def predict_ontram_ensemble(res_tab, X, tabular_df, model_names, cnn):

    X = X.reshape(1,128,128,28,1)
    
    out = []
    for model_name in model_names:
        cnn.load_weights(model_name)

        filtered_df = tabular_df[tabular_df['p_id'] == res_tab['p_id'][0]].drop('p_id', axis=1).values
        X_tab_occ = np.tile(filtered_df, (len(X), 1))

        occ_dataset_pred = ((X, X_tab_occ))
        preds = cnn.predict(occ_dataset_pred)
        out.append(preds[:,0]-preds[:,1])
    
    return 1-fm.sigmoid(np.mean((out)))

In [ ]:
predict_ontram_ensemble(res_table, resized_img, pat_norm_tab_cp, res_model_names[0][:], model_3d)

In [ ]:
start = time.time()
(heatmap, resized_img, max_hm_slice, hm_mean_std, all_heatmaps) =  oc.volume_occlusion(
    volume = res_images, 
    res_tab = res_table, 
    tabular_df=pat_norm_tab_cp,
    occlusion_size = np.array(occ_size), 
    normalize = False,
    both_directions = False,
    invert_hm = "pred_class",
    cnn = model_3d,
    model_names = res_model_names[0][:], # select model
    occlusion_stride = occ_stride,
    reset_cut_off=True)

end = time.time()
print(end - start)

In [ ]:
phm.plot_heatmap(np.squeeze(resized_img, axis=-1), np.squeeze(heatmap, axis=-1),
                version = "overlay",
                mode = "avg",
                hm_colormap="jet",
                hm_positive=True,
                colorbar=True,
                )

In [ ]:
phm.plot_heatmap(np.squeeze(resized_img, axis=-1), np.squeeze(heatmap, axis=-1),
                version = "overlay",
                mode = "def",
                slices = (50,20,20),
                hm_colormap="jet",
                hm_positive=True,
                colorbar=True,
                add_orientation=True)

# Occlusion Slider

In [ ]:
## old occlusion
# occ_size = (16, 16, 12)
# occ_stride = 8
# occ_size = (14, 14, 10)
# occ_stride = 6

## new occlusion
# occ_size = (20, 20, 16)
# occ_stride = 6
occ_size = (18, 18, 4)
occ_stride = (10, 10, 3)

sl.occlusion_interactive_plot(
    12, # patient id
    occ_size = occ_size, occ_stride = occ_stride,
    cnn=model_3d, all_results=all_results_tab, pat=pat_ids, X_in=X_in,
    generate_model_name=generate_model_name, num_models=num_models,
    pat_dat=pat_orig_tab, pat_norm_table=pat_norm_tab,
    normalize_hm=False,
    pred_hm_only=True) # if True, only the heatmap for the predicted class is shown, otherwise also negative heatmaps are shown